# 6-DaskCudaDataframe

This notebook demonstrates how to read and process a tabular datafile with the [DASK-cuDF](https://docs.rapids.ai/api/cudf/latest/user_guide/10min.html) dataframe library. A Dask-cuDF dataframe is a large parallel dataframe composed of smaller cuDF GPU dataframes. Dask-cuDF can handle datasets that are large than the available GPU memory (out-of-core) and process them on one or multiple GPUs.

Author: Peter W. Rose (pwrose@ucsd.edu)

In [ ]:
import os
import pandas as pd
from dask.distributed import Client
import dask_cudf
import time

If LOCAL_SCRATCH_DIR environment variable is not set, this notebook accesses the ../data directory for temporary files.

In [ ]:
DATA_DIR = os.getenv("LOCAL_SCRATCH_DIR", default="../data")
filename = os.path.join(DATA_DIR, "gene_info.tsv")
file_size = f"{os.path.getsize(filename)/1E9:.1f}"
RESULTS_DIR = "results"
os.makedirs(RESULTS_DIR, exist_ok=True)

### Setup Benchmark

The ```n_cores``` and ```file_format``` parameter are used for benchmarking ([see](7-ParallelEfficiency.ipynb)). 
The cell below has been [parameterized](https://papermill.readthedocs.io/en/latest/usage-parameterize.html#jupyterlab-3-0) as input parameters for [papermill](https://papermill.readthedocs.io/en/latest/index.html).

In [ ]:
n_cores = 0 # default -> uses all available cores
file_format = "csv"

In [ ]:
start = time.time()

### Read Data

In [ ]:
if n_cores > 0:
    # use n_cores for benchmarking
    client = Client(n_workers=n_cores, threads_per_worker=1)
else:
    # use all available cores
    client = Client()

In [ ]:
# read only specified columns and rows
column_names = ["GeneID", "Symbol", "Synonyms", "description", "type_of_gene", "#tax_id", "chromosome"]
filters=[[("type_of_gene", "==", "protein-coding")]]

if file_format == "csv":
    filename = os.path.join(DATA_DIR, "gene_info.tsv")
    genes = dask_cudf.read_csv(filename, usecols=column_names, dtype=str, sep="\t")
    genes = genes[genes["type_of_gene"] == 'protein-coding']
elif file_format == "parquet":
    filename = os.path.join(DATA_DIR, "gene_info.parquet")
    #genes = dask_cudf.read_parquet(filename, columns=column_names, filters=filters) # filters are ignored
    genes = dask_cudf.read_parquet(filename, columns=column_names)
    genes = genes[genes["type_of_gene"] == "protein-coding"]
else:
    print("invalid file format")
    
print("Filename:", filename)

genes = genes.rename(columns={"#tax_id": "tax_id"})

### Process Data

In [ ]:
groups = genes.groupby("tax_id").size().reset_index()
groups.columns = ["tax_id", "count"]
groups = groups.sort_values("count", ascending=False)

Convert cuDF to Pandas dataframe (this triggers the computation)

In [ ]:
groups = groups.compute()

In [ ]:
# terminate the Dask processes
client.close()

### Display Results

#### Number of human protein-coding genes (tax_id = 9606)

In [ ]:
groups[groups["tax_id"]  == "9606"]

#### Top 5 organisms with the most protein-coding genes

In [ ]:
groups.head()

In [ ]:
end = time.time()

In [ ]:
if n_cores > 0:
    df = pd.DataFrame([{"cores": n_cores, "time": end-start, "size": file_size, "format": file_format, "dataframe": "DaskCuda"}])
    output_file = f"6-DaskCudaDataframe_{file_size}_{file_format}_{n_cores}.csv"
    df.to_csv(os.path.join(RESULTS_DIR, output_file), index=False)

In [ ]:
print(f"Dask-cuDF: {end - start:.1f} sec.")